In [30]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import warnings
warnings.filterwarnings('ignore')

In [8]:
# Load Excel file
file_path = 'Data_preparation_w_e.xlsx'  # Update if using a different path
excel_data = pd.ExcelFile(file_path)


In [9]:
# Load each sheet
weather_df = excel_data.parse('weather data')
event_df = excel_data.parse('Event data')
traffic_df = excel_data.parse('traffic data')

In [10]:
weather_df.head()

,DateTime,date,DateTime.1,Unnamed: 3,temp,humidity,preciptype,windspeed
0,2015-11-01 00:00:00,NaN,2015-11-01,00:00:00,22.444444,71.7,0,12.8
1,2015-11-01 01:00:00,NaN,2015-11-01,01:00:00,22.444444,71.7,0,12.8
2,2015-11-01 02:00:00,NaN,2015-11-01,02:00:00,22.444444,71.7,0,12.8
3,2015-11-01 03:00:00,NaN,2015-11-01,03:00:00,22.444444,71.7,0,12.8
4,2015-11-01 04:00:00,NaN,2015-11-01,04:00:00,22.444444,71.7,0,12.8


In [13]:
weather_df = weather_df.drop(columns=['date','DateTime.1'], axis=1)

In [14]:
weather_df.head()

,DateTime,Unnamed: 3,temp,humidity,preciptype,windspeed
0,2015-11-01 00:00:00,00:00:00,22.444444,71.7,0,12.8
1,2015-11-01 01:00:00,01:00:00,22.444444,71.7,0,12.8
2,2015-11-01 02:00:00,02:00:00,22.444444,71.7,0,12.8
3,2015-11-01 03:00:00,03:00:00,22.444444,71.7,0,12.8
4,2015-11-01 04:00:00,04:00:00,22.444444,71.7,0,12.8


In [18]:
weather_df= weather_df[['DateTime','temp','humidity','preciptype','windspeed']]


In [19]:
weather_df.head()

,DateTime,temp,humidity,preciptype,windspeed
0,2015-11-01 00:00:00,22.444444,71.7,0,12.8
1,2015-11-01 01:00:00,22.444444,71.7,0,12.8
2,2015-11-01 02:00:00,22.444444,71.7,0,12.8
3,2015-11-01 03:00:00,22.444444,71.7,0,12.8
4,2015-11-01 04:00:00,22.444444,71.7,0,12.8


In [20]:
event_df.head()

,DateTime,event_name,category,location,expected_traffic_impact
0,2015-11-01 00:00:00,,,,
1,2015-11-01 01:00:00,,,,
2,2015-11-01 02:00:00,,,,
3,2015-11-01 03:00:00,,,,
4,2015-11-01 04:00:00,,,,


In [21]:
traffic_df.head()

,DateTime,Junction,Vehicles,ID
0,2015-11-01 00:00:00,1,15,20151101001
1,2015-11-01 01:00:00,1,13,20151101011
2,2015-11-01 02:00:00,1,10,20151101021
3,2015-11-01 03:00:00,1,7,20151101031
4,2015-11-01 04:00:00,1,9,20151101041


In [22]:
# Fill missing values in Event data
event_df.fillna({
    'event_name': 'None',
    'category': 'None',
    'location': 'None',
    'expected_traffic_impact': 'None'
}, inplace=True)

In [23]:
# Merge traffic with weather
traffic_weather_df = pd.merge(
    traffic_df,
    weather_df[['DateTime', 'temp', 'humidity', 'preciptype', 'windspeed']],
    on='DateTime',
    how='left'
)

In [25]:
# Merge with event data
unified_df = pd.merge(
    traffic_weather_df,
    event_df[['DateTime', 'event_name', 'category', 'location', 'expected_traffic_impact']],
    on='DateTime',
    how='left')

In [27]:
unified_df.head()

,DateTime,Junction,Vehicles,ID,temp,humidity,preciptype,windspeed,event_name,category,location,expected_traffic_impact
0,2015-11-01,1,15,20151101001,22.444444,71.7,0,12.8,,,,
1,2015-11-01,1,15,20151101001,22.444444,71.7,0,12.8,,,,
2,2015-11-01,1,15,20151101001,22.444444,71.7,0,12.8,,,,
3,2015-11-01,1,15,20151101001,22.444444,71.7,0,12.8,,,,
4,2015-11-01,1,15,20151101001,22.444444,71.7,0,12.8,,,,


In [31]:
#Convert 'DateTime' to datetime
weather_df['DateTime'] = pd.to_datetime(weather_df['DateTime'])
event_df['DateTime'] = pd.to_datetime(event_df['DateTime'])
traffic_df['DateTime'] = pd.to_datetime(traffic_df['DateTime'])

In [32]:
# Remove duplicates
unified_df.drop_duplicates(inplace=True)

In [33]:
# Drop rows with missing DateTime
unified_df.dropna(subset=['DateTime'], inplace=True)

In [34]:
# Fill missing numeric values with median
for col in ['temp', 'humidity', 'windspeed', 'Vehicles']:
    if col in unified_df.columns:
        unified_df[col].fillna(unified_df[col].median(), inplace=True)

In [35]:
# Fill missing categorical values with 'Unknown'
categorical_cols = ['preciptype', 'event_name', 'category', 'location', 'expected_traffic_impact']
for col in categorical_cols:
    if col in unified_df.columns:
        unified_df[col].fillna('Unknown', inplace=True)

In [36]:
scaler = MinMaxScaler()
numeric_cols = ['temp', 'humidity', 'windspeed', 'Vehicles']
unified_df[numeric_cols] = scaler.fit_transform(unified_df[numeric_cols])
